In [9]:
import pytest
import numpy as np
from astropy.io import fits
import os
import os
import numpy as np
import nitrates_response as nr

In [10]:
def get_bldmask_alldets():

    detxs_by_sand0 = np.arange(0, 286-15, 18)
    detxs_by_sand1 = detxs_by_sand0 + 15

    detys_by_sand0 = np.arange(0, 173-7, 11)
    detys_by_sand1 = detys_by_sand0 + 7

    all_good_detxs = np.ravel([np.arange(detxs_by_sand0[i], detxs_by_sand1[i]+1,\
                                          1, dtype=np.int) for i in range(16)])
    all_good_detys = np.ravel([np.arange(detys_by_sand0[i], detys_by_sand1[i]+1,\
                                         1, dtype=np.int) for i in range(16)])

    detxax = np.arange(286, dtype=np.int)
    detyax = np.arange(173, dtype=np.int)
    detx_dpi, dety_dpi = np.meshgrid(detxax, detyax)
    bl_alldets = np.isin(detx_dpi, all_good_detxs)&np.isin(dety_dpi, all_good_detys)
    return bl_alldets


Constructing the energy bins for the responses and storing them as ebins, with values ranging from 15.0keV up to 500keV. The values in ebins0 represent the lower ends of each energy bin and ebins1 the upper ends.

In [11]:
ebins0 = np.array([15.0, 24.0, 35.0, 48.0, 64.0])
ebins0 = np.append(ebins0, np.logspace(np.log10(84.0), np.log10(500.0), 5+1))[:-1]
ebins0 = np.round(ebins0, decimals=1)[:-1]
ebins1 = np.append(ebins0[1:], [350.0])
ebins = [ebins0, ebins1]

We'll gather the response using the following values for the theta and phi positions.

In [12]:
th = 0.
ph = 0.

In order to generate the response, we have to initialize a flux model based on a cutoff power law flux model in addition to a RayTraces object, rt_obj.

In [13]:
flux_mod = nr.models.flux_models.Cutoff_Plaw_Flux(E0=100.0)
bl_alldets = get_bldmask_alldets()
rt_obj = nr.response.ray_trace_funcs.RayTraces(nr.config.rt_dir)

/var/folders/_4/y52q1x_130bg2r9mb5ctl5340000gn/T/ipykernel_64979/587707886.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  1, dtype=np.int) for i in range(16)])
/var/folders/_4/y52q1x_130bg2r9mb5ctl5340000gn/T/ipykernel_64979/587707886.py:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cur

With the above defined, we can now initiate our signal model using the Source_Model_InOutFoV object from the models.py module. 

In [14]:
sig_mod = nr.models.models.Source_Model_InOutFoV(flux_mod, ebins,\
                                        bl_alldets, rt_obj, use_deriv=True,\
                                        resp_tab_dname=nr.config.resp_tabs)

We must run .set_theta_phi() using the values of *th* and *ph* in order to generate the appropriate response for our desired coordinates. With the coordinates set, we can calculate the total response dpi.

In [15]:
sig_mod.set_theta_phi(th, ph)
tot_resp_dpis = sig_mod.resp_obj.lines_resp_dpi + sig_mod.resp_obj.comp_flor_resp_dpis

(0.0, 30.48, -14.117)
(0.0, 30.48, -14.117)
(0.0, 3.0)
0.0
0.0
max rt: 1.0000
initing ResponseDPI, with fname
/Users/daniel/Development/Astronomy/UnmaskedResponder/resp_tabs/drm_theta_0.0_phi_0.0_.fits


In [16]:
tot_resp_dpis

array([[[1.07772476e-04, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [8.92803152e-04, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.37674939e-03, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.78259835e-04, 2.66266596e-04, 3.09591165e-04, ...,
         1.04871326e-03, 1.46304546e-03, 2.00889555e-03],
        [1.08136724e-04, 1.68412922e-04, 1.94899718e-04, ...,
         7.93911238e-04, 1.14881267e-03, 1.60499934e-03],
        [7.33258100e-05, 1.17401120e-04, 1.29014299e-04, ...,
         5.75196290e-04, 8.39456518e-04, 1.17993682e-03]],

       [[1.08845206e-04, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [9.01689805e-04, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.42031403e-03, 